# Weather Planner – Service Tests (ohne MCP)

Dieses Notebook testet die **reinen Services** aus `weather_planner/services/`.

- Geocoding (Nominatim)
- Wetter (Open-Meteo)
- POIs (Overpass)
- Scoring/Ranking (Heuristik)

Hinweise:
- Alle APIs sind **tokenfrei**, aber **rate-limited** → Cache ist standardmäßig aktiv.
- Wenn Overpass langsam ist, kann ein zweiter Run dank Cache deutlich schneller sein.


In [1]:
# Wenn du dieses Notebook im Projekt-Root ausführst, sollte der Import direkt funktionieren.
# Falls nicht (z.B. weil du das Notebook woanders öffnest), setze den Root-Pfad explizit:
import os, sys, pathlib

ROOT = pathlib.Path().resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)


Project root: /home/simon/KI-Workshop_2/weather_planner_mcp_tools


In [2]:
from mcp_tools.core.cache import FileCache
from mcp_tools.core.schemas import Coordinates, TripSpec, WeatherDay, WeatherProfile, Spot
from mcp_tools.services.geocoding import geocode_destination
from mcp_tools.services.weather import get_weather_profile
from mcp_tools.services.pois import get_activity_spots
from mcp_tools.services.scoring import score_spots_heuristic

from mcp_tools.utils.geo import haversine_km, travel_time_minutes


## 1) Cache initialisieren

Wir legen den Cache in `./data/cache` an (relativ zum Projekt-Root).

In [3]:
cache_dir = "./data/cache"
cache = FileCache(cache_dir)
cache_dir


'./data/cache'

## 2) Distance und Timecalculation testen


In [4]:
destination_A = "Barcelona"
destination_B = "Palma"

coords_A = geocode_destination(destination_A, cache=cache)
coords_B = geocode_destination(destination_B, cache=cache)

dist = haversine_km(coords_A.lat, coords_A.lon, coords_B.lat, coords_B.lon)

travel_time = travel_time_minutes(dist)

print(f"Distance destination_A:{destination_A}, destination_B:{destination_B} = {dist} km travel_time:{travel_time} minutes")

Distance destination_A:Barcelona, destination_B:Palma = 205.52677543399773 km travel_time:205.52677543399773 minutes


## 3) Geocoding testen (Nominatim)

Wir geocoden eine Zielregion. Du kannst das frei ändern (z.B. `Barcelona`, `Toskana`, `Mittelmeer`).

In [5]:
destination = "Barcelona"
coords = geocode_destination(destination, cache=cache)

if isinstance(coords, Coordinates):
    print("Coordinates bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(coords))

print(coords)


Coordinates bestätigt:
lat=41.3825802 lon=2.177073


## 4) Wetter testen forecast (Open-Meteo bis 16 Tage)

Wir holen das tägliche Wetterprofil für einen Zeitraum.

> Tipp: Für Trainings ist es oft hilfreich, nahe Zukunftsdaten zu nehmen, um "realistische" Forecasts zu sehen.

In [8]:
from datetime import date, timedelta

start = date.today() + timedelta(days=1)
end   = start + timedelta(days=7)

weather = get_weather_profile(coords, start, end, cache=cache, include_raw=False)

if isinstance(weather, WeatherProfile):
    print("WeatherProfile bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(weather))

weather


WeatherProfile bestätigt:


WeatherProfile(start_date=datetime.date(2026, 1, 16), end_date=datetime.date(2026, 1, 23), days=[WeatherDay(date=datetime.date(2026, 1, 16), temp_min_c=5.3, temp_max_c=13.9, precipitation_mm=0.0, wind_max_kmh=7.7, weather_code=45), WeatherDay(date=datetime.date(2026, 1, 17), temp_min_c=7.9, temp_max_c=13.1, precipitation_mm=32.1, wind_max_kmh=15.8, weather_code=95), WeatherDay(date=datetime.date(2026, 1, 18), temp_min_c=10.8, temp_max_c=13.6, precipitation_mm=26.8, wind_max_kmh=15.1, weather_code=95), WeatherDay(date=datetime.date(2026, 1, 19), temp_min_c=9.3, temp_max_c=12.6, precipitation_mm=25.9, wind_max_kmh=18.5, weather_code=80), WeatherDay(date=datetime.date(2026, 1, 20), temp_min_c=10.5, temp_max_c=12.4, precipitation_mm=27.1, wind_max_kmh=33.7, weather_code=63), WeatherDay(date=datetime.date(2026, 1, 21), temp_min_c=7.5, temp_max_c=12.8, precipitation_mm=0.0, wind_max_kmh=17.7, weather_code=3), WeatherDay(date=datetime.date(2026, 1, 22), temp_min_c=4.2, temp_max_c=13.6, precip

In [9]:
import pandas as pd

df_weather = pd.DataFrame([d.model_dump() for d in weather.days])
print(df_weather)

         date  temp_min_c  temp_max_c  precipitation_mm  wind_max_kmh  \
0  2026-01-16         5.3        13.9               0.0           7.7   
1  2026-01-17         7.9        13.1              32.1          15.8   
2  2026-01-18        10.8        13.6              26.8          15.1   
3  2026-01-19         9.3        12.6              25.9          18.5   
4  2026-01-20        10.5        12.4              27.1          33.7   
5  2026-01-21         7.5        12.8               0.0          17.7   
6  2026-01-22         4.2        13.6               0.0           9.5   
7  2026-01-23         7.2        12.3               0.0           8.9   

   weather_code  
0            45  
1            95  
2            95  
3            80  
4            63  
5             3  
6             3  
7             1  


### Wetter-Summary (Aggregationen)

Diese Aggregationen werden später für Scoring, Packing-List und Heuristiken genutzt.

In [11]:
{
    "temp_min_c": weather.temp_min_c,
    "temp_max_c": weather.temp_max_c,
    "precip_total_mm": weather.precip_total_mm,
    "rainy_days": weather.rainy_days,
    "wind_max_kmh": weather.wind_max_kmh,
}

{'temp_min_c': None,
 'temp_max_c': None,
 'precip_total_mm': None,
 'rainy_days': None,
 'wind_max_kmh': None}

## 5) Wetter testen historical (Open-Meteo ab 17 Tage)

Wir holen das tägliche Wetterprofil für einen Zeitraum.

In [12]:
# Far-future range: triggers historical fallback built from archive data
future_start = date(date.today().year + 1, 6, 10)
future_end   = date(date.today().year + 1, 6, 17)

weather_future = get_weather_profile(coords, future_start, future_end, cache=cache, include_raw=False)

if isinstance(weather_future, WeatherProfile):
    print("WeatherProfile bestätigt:")
else:
    print("Ergebnis hat anderen Typ:", type(weather_future))

print('source:', weather_future.source, '| estimate:', weather_future.is_estimate)
print('reference_years (sample):', weather_future.reference_years[:5])
weather_future


WeatherProfile bestätigt:
source: historical_fallback | estimate: True
reference_years (sample): [2025, 2024, 2023, 2022, 2021]


WeatherProfile(start_date=datetime.date(2027, 6, 10), end_date=datetime.date(2027, 6, 17), days=[WeatherDay(date=datetime.date(2027, 6, 10), temp_min_c=17.68, temp_max_c=25.78, precipitation_mm=0.71, wind_max_kmh=17.509999999999998, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 11), temp_min_c=18.02, temp_max_c=25.62, precipitation_mm=4.859999999999999, wind_max_kmh=16.12, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 12), temp_min_c=18.19, temp_max_c=25.56, precipitation_mm=1.5, wind_max_kmh=19.51, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 13), temp_min_c=17.95, temp_max_c=26.28, precipitation_mm=1.79, wind_max_kmh=18.68, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 14), temp_min_c=18.28, temp_max_c=26.42, precipitation_mm=1.18, wind_max_kmh=19.3, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 15), temp_min_c=18.89, temp_max_c=26.41, precipitation_mm=0.02, wind_max_kmh=17.79, weather_code=3), WeatherDay(date=datetime.date(2027, 6, 

In [13]:
import pandas as pd

df_weather_future = pd.DataFrame([d.model_dump() for d in weather_future.days])
print(df_weather_future)


         date  temp_min_c  temp_max_c  precipitation_mm  wind_max_kmh  \
0  2027-06-10       17.68       25.78              0.71         17.51   
1  2027-06-11       18.02       25.62              4.86         16.12   
2  2027-06-12       18.19       25.56              1.50         19.51   
3  2027-06-13       17.95       26.28              1.79         18.68   
4  2027-06-14       18.28       26.42              1.18         19.30   
5  2027-06-15       18.89       26.41              0.02         17.79   
6  2027-06-16       18.71       26.60              0.48         18.88   
7  2027-06-17       19.06       25.98              0.38         18.46   

   weather_code  
0             3  
1             3  
2             3  
3             3  
4             3  
5             3  
6             3  
7             3  


### Wetter-Summary (Aggregationen)

Diese Aggregationen werden später für Scoring, Packing-List und Heuristiken genutzt.

In [14]:
{
    "temp_min_c": weather_future.temp_min_c,
    "temp_max_c": weather_future.temp_max_c,
    "precip_total_mm": weather_future.precip_total_mm,
    "rainy_days": weather_future.rainy_days,
    "wind_max_kmh": weather_future.wind_max_kmh,
}


{'temp_min_c': 17.68,
 'temp_max_c': 26.6,
 'precip_total_mm': 10.92,
 'rainy_days': 4,
 'wind_max_kmh': 19.51}

## 6) POI/Spots testen (Overpass)

Wir ziehen Spots passend zu einer Aktivität und einem Suchradius.

In [ ]:
from typing import List

activity = "sightseeing"   # z.B. hiking, running, beach, sightseeing
radius_km = 10.0

spots = get_activity_spots(coords.lat, coords.lon, radius_km, activity, cache=cache)

if isinstance(spots, list):
    print("Liste bestätigt")
    if all(isinstance(s, Spot) for s in spots):
        print("und die Elemente sind Spot-Instanzen")
    else:
        print("aber die Elemente haben anderen Typ:", {type(s) for s in spots})
else:
    print("kein list:", type(spots))

len(spots), spots[:3]


In [ ]:
df_spots = pd.DataFrame([{"name": s.name, "lat": s.lat, "lon": s.lon, "activity": s.activity} for s in spots])
df_spots.head(10)


## 7) Scoring/Ranking testen

Wir berechnen Distanz/Travel-Time + Heuristik-Score und sortieren absteigend.

> Später ersetzt/ergänzt ein ML-Modell die Heuristik – aber die Schnittstelle bleibt gleich.

In [ ]:
ranked = score_spots_heuristic(coords.lat, coords.lon, spots, weather)

if isinstance(spots, list):
    print("Liste bestätigt")
    if all(isinstance(s, Spot) for s in spots):
        print("und die Elemente sind Spot-Instanzen")
    else:
        print("aber die Elemente haben anderen Typ:", {type(s) for s in spots})
else:
    print("kein list:", type(spots))

ranked[:5]


In [ ]:
df_ranked = pd.DataFrame([
    {
        "name": s.name,
        "activity": s.activity,
        "distance_km": round(s.distance_km or 0, 2),
        "travel_time_min": round(s.travel_time_min or 0, 1),
        "score": round(s.score or 0, 1),
    }
    for s in ranked[:15]
])
df_ranked


## 8) Mini-End-to-End: mehrere Aktivitäten

Optional: Für mehrere Aktivitäten nacheinander ziehen und die Top-Spots ansehen.


In [ ]:
activities = ["sightseeing", "running"]  # z.B. ["beach","hiking","sightseeing"]
radius_km = 12.0
top_k = 5

results = {}
for act in activities:
    s = get_activity_spots(coords.lat, coords.lon, radius_km, act, cache=cache)
    r = score_spots_heuristic(coords.lat, coords.lon, s, weather)[:top_k]
    results[act] = r

for act, r in results.items():
    print("\n===", act, "===")
    for item in r:
        print(f"- {item.name} | {item.score:.1f} | {item.distance_km:.2f} km")
